In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

In [2]:
DATA_DIR="E:\Python\Spoofing-Version02\images"
CATEGORIES=["fake","real"]

In [3]:
IMG_SIZE=160

In [4]:
training_data=[]
def create_training_data():
    for category in CATEGORIES:
        path=os.path.join(DATA_DIR,category) #path to cats and dogs
        class_num=CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_arr=cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
                new_array=cv2.resize(img_arr,(IMG_SIZE,IMG_SIZE))
                training_data.append([new_array,class_num])
            except Exception as e:
                pass
    
create_training_data()

In [5]:
print(len(training_data))

6389


In [121]:
import pickle
pickle_out=open("training_data.pickle","wb")
pickle.dump(training_data,pickle_out)
pickle_out.close()

In [6]:

import random

random.shuffle(training_data)

In [7]:
for sample in training_data[:5]:
    print(sample[1])

1
1
1
1
0


In [8]:
X=[]
y=[]

for features,label in training_data:
    X.append(features)
    y.append(label)
    
#how many features did we have -1 saying it could be many any number,we know shape of data imgsize x imgsize
#and than one bcz of grayscale image
#LAST 1 WILL BE 3 IF USING RGB
X=np.array(X).reshape(-1,IMG_SIZE,IMG_SIZE,1)
print(X.shape)

(6389, 160, 160, 1)


In [9]:
print(len(X))
print(len(y))

6389
6389


In [10]:
X=X/255.0

In [11]:
X_train=X[:4999]
y_train=y[:4999]
X_test=X[4999:]
y_test=y[4999:]
print(X_train.shape)

(4999, 160, 160, 1)


In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten,Conv2D,MaxPooling2D,Conv3D,MaxPooling3D

In [13]:
model = Sequential()
model.add(Conv2D(filters=64, kernel_size=3, input_shape=X.shape[1:]))

model.add(Conv2D(64,kernel_size=5, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=128,kernel_size=5, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
          
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(64, activation='relu'))

model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 158, 158, 64)      640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 154, 154, 64)      102464    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 77, 77, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 73, 73, 128)       204928    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 128)       0         
_________________________________________________________________
dropout (Dropout)            (None, 36, 36, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 165888)            0

In [14]:
#model.compile(loss="binary_crossentropy",optimizer="adam",metrics=['accuracy'])
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=['accuracy'])

In [15]:
model.fit(X_train,np.array(y_train),batch_size=5,epochs=2,validation_split=0.2)

Train on 3999 samples, validate on 1000 samples
Epoch 1/2
3999/3999 [==============================] - 2845s 711ms/sample - loss: 0.1129 - accuracy: 0.9682 - val_loss: 7.3000e-05 - val_accuracy: 1.0000
Epoch 2/2
3999/3999 [==============================] - 2951s 738ms/sample - loss: 0.0126 - accuracy: 0.9965 - val_loss: 6.7151e-05 - val_accuracy: 1.0000


In [16]:
val_loss,val_accuracy=model.evaluate(X_test,np.array(y_test))

1390/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [17]:
import h5py
model.save('CNN_model.h5')

In [18]:
import pickle

In [19]:
pickle_out=open("X_train.pickle","wb")
pickle.dump(X_train,pickle_out)
pickle_out.close()

pickle_out=open("X_test.pickle","wb")
pickle.dump(X_test,pickle_out)
pickle_out.close()

pickle_out=open("y_train.pickle","wb")
pickle.dump(y_train,pickle_out)
pickle_out.close()

pickle_out=open("y_test.pickle","wb")
pickle.dump(y_test,pickle_out)
pickle_out.close()


In [20]:
n=1
print(y_test[n])

1


In [21]:
pred = model.predict([X_test])

In [119]:
n=0
print(y_test[n])

1


In [120]:
#print(np.argmax(pred[n]))
print(pred[n])

[8.73606e-05]


In [59]:
def func(pred):
    if pred==0:
        return "fake"
    else :
        return "real"
    

In [136]:
IMG_SIZE=160
def CNN_frame_shape(img):
    img=cv2.resize(img,(IMG_SIZE,IMG_SIZE))
    img_new = img.reshape((-1,img.shape[0],img.shape[1],1))
    return img_new


In [274]:
#my_img=cv2.imread("E:\Python\liveness_detection\Hannan-13.jpeg",cv2.IMREAD_GRAYSCALE)
#my_img=cv2.imread("E:\Python\liveness_detection\mahadFriend-Fake.jpeg",cv2.IMREAD_GRAYSCALE)
#my_img=cv2.imread("E:\Python\Spoofing\Wasif-02.jpg",cv2.IMREAD_GRAYSCALE)
#my_img=cv2.imread("E:\Python\liveness_detection\justin.jpg",cv2.IMREAD_GRAYSCALE)
#my_img=cv2.imread(r"E:\Python\Spoofing-Version02\images\fake\Old Hannan576.jpg",cv2.IMREAD_GRAYSCALE)
my_img=cv2.imread("E:\Python\Spoofing-Version02\Fake-Hannan.jpeg",cv2.IMREAD_GRAYSCALE)
#my_img=cv2.imread(r"E:\Python\liveness_detection\images\fake\img1.jpg",cv2.IMREAD_GRAYSCALE)

In [275]:
roi = cv2.resize(my_img, (160, 160))
#print(roi)
roi=roi/255

In [276]:
f_img=CNN_frame_shape(roi)
print(f_img.shape)
pred = model.predict(f_img)
print(pred)
var=func(pred)
print(var)

(1, 160, 160, 1)
[[0.]]
fake


In [277]:
pred[0][0]

0.0

In [278]:
def func_CNN(pred):
    chk=pred[0][0]
    if chk < 0.01:
        result = "fake"
    else:
        result = "real"
    return result

In [282]:
#my_img2=cv2.imread("E:\Python\liveness_detection\mahadFriend-Fake.jpeg",cv2.IMREAD_GRAYSCALE)
#my_img2=cv2.imread("E:\Python\Spoofing-Version02\Fake-Hannan.jpeg",cv2.IMREAD_GRAYSCALE)
#my_img2=cv2.imread(r"E:\Python\liveness_detection\images\fake\img1.jpg",cv2.IMREAD_GRAYSCALE)
my_img2=cv2.imread("E:\Python\Spoofing\Wasif-02.jpg",cv2.IMREAD_GRAYSCALE)

In [283]:
roi = cv2.resize(my_img2, (160, 160))
#print(roi)
roi=roi/255

In [284]:
f_img=CNN_frame_shape(roi)
print(f_img.shape)
pred = model.predict(f_img)
print(pred)
var=func_CNN(pred)
print(var)

(1, 160, 160, 1)
[[0.01333151]]
real
